## setup

In [2]:
#import packages
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import colorcet as cc
from colorcet.plotting import swatch, swatches, candy_buttons
import warnings
warnings.simplefilter('ignore',  xr.SerializationWarning)

In [3]:
#copy your local directory here
os.chdir('C:/Users/elayton/Desktop/RFMIP/RFMIP-rad-irf-intake-main')

In [4]:
#open the catalog and experiment info
from intake import open_catalog
cat = open_catalog(os.getcwd() + '/main.yml')
extrainfo = xr.open_dataset(os.getcwd() + '/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc').to_dataframe()

In [5]:
#flip RRTMG p1f1 data since it is upside-down
#concatenate the RRTMG data
original_sw_rrtmg = xr.concat([cat.parameterized.RRTMG.p1f1[f].to_dask()
                for f in ["rsu","rsd"]],
                dim="field", data_vars = "different")
corrected_rrtmg = xr.concat([cat.parameterized.RRTMG.p1f1[f].to_dask()
                for f in ["rsu","rsd","rlu","rld"]],
                dim="field", data_vars = "different")
corrected_rrtmg['rsd'] = np.flip(original_sw_rrtmg['rsd'], 2)
corrected_rrtmg['rsu'] = np.flip(original_sw_rrtmg['rsu'], 2)

In [6]:
#method to compute weighted global mean
def compute_global_mean(ds):
    wts = ds.profile_weight/ds.profile_weight.sum()
    return(ds.weighted(wts).mean(["site"]))

In [7]:
#make a dataset of the pre-averaged parameterized model values
un_avgd = {}
for m in list(cat.parameterized):
    for r in list(cat.parameterized[m]):
        components = list(cat.parameterized[m][r])
        un_avgd[f'{m}_{r}'] = xr.concat([(cat.parameterized[m][r][f].to_dask()) 
                                        for f in components], 
                                        dim="field", data_vars = "different")
        if "rsd" in components:
            un_avgd[f'{m}_{r}'] = un_avgd[f'{m}_{r}'].assign(rsn=un_avgd[f'{m}_{r}'].data_vars['rsd']-un_avgd[f'{m}_{r}'].data_vars['rsu'])

un_avgd = {k: un_avgd[k].assign(rln=un_avgd[k].data_vars['rld']-un_avgd[k].data_vars['rlu']) for k in un_avgd.keys()}
un_avgd = {k: un_avgd[k].assign_coords(expt=extrainfo['expt_label'].unique()) for k in un_avgd.keys()}



In [8]:
#make a dataset of the pre-averaged parameterized model values
un_avgd_b = {}
for m in list(cat.benchmark):
    if 'fourAOP' not in m:
        for r in list(cat.benchmark[m]):
            components = list(cat.benchmark[m][r])
            un_avgd_b[f'{m}_{r}'] = xr.concat([(cat.benchmark[m][r][f].to_dask()) 
                                            for f in components], 
                                            dim="field", data_vars = "different")
            if "rsd" in components:
                un_avgd_b[f'{m}_{r}'] = un_avgd_b[f'{m}_{r}'].assign(rsn=un_avgd_b[f'{m}_{r}'].data_vars['rsd']-un_avgd_b[f'{m}_{r}'].data_vars['rsu'])

un_avgd_b = {k: un_avgd_b[k].assign(rln=un_avgd_b[k].data_vars['rld']-un_avgd_b[k].data_vars['rlu']) for k in un_avgd_b.keys()}
un_avgd_b = {k: un_avgd_b[k].assign_coords(expt=extrainfo['expt_label'].unique()) for k in un_avgd_b.keys()}


## probe the difference between unaveraged parameterized and LBLRTM rsd values at TOA

In [14]:
#compute the difference in globally averaged rsd at TOA between a sample parameterized model and LBLRTM p1f1 for 18 experiments
TOA=0
(compute_global_mean(un_avgd['CanESM5_p2f2'])-compute_global_mean(un_avgd_b['LBLRTM_p1f1'])).rsd.sel(level=TOA)

<xarray.DataArray 'rsd' (expt: 18)>
array([0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551,
       0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551,
       0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551],
      dtype=float32)
Coordinates:
  * expt     (expt) object 'Present day (PD)' ... 'LGM'

In [15]:
#another sample parameterized model, just to check
(compute_global_mean(un_avgd['MIROC6_p1f1'])-compute_global_mean(un_avgd_b['LBLRTM_p1f1'])).rsd.sel(level=TOA)

<xarray.DataArray 'rsd' (expt: 18)>
array([0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551,
       0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551,
       0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551, 0.5427551],
      dtype=float32)
Coordinates:
  * expt     (expt) object 'Present day (PD)' ... 'LGM'

In [19]:
#since the error is the same across all experiments, focus on experiment 1 (present day (PD))
#look at the difference in unaveraged values
print((un_avgd['CanESM5_p2f2'].rsd.values-un_avgd_b['LBLRTM_p1f1'].rsd.values)[0,:,0])

[-3.8452148e-03 -4.0893555e-03  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00 -6.3476562e-03  0.0000000e+00
 -1.8310547e-03  0.0000000e+00 -1.7089844e-03 -6.7138672e-04
 -1.5563965e-03  0.0000000e+00 -2.2888184e-03 -5.9814453e-03
  0.0000000e+00  0.0000000e+00 -1.0223389e-03  0.0000000e+00
 -2.4108887e-03 -4.1503906e-03 -4.8828125e-03  0.0000000e+00
  0.0000000e+00 -2.9907227e-03 -6.2255859e-03 -4.1503906e-03
 -5.3710938e-03  0.0000000e+00 -3.2958984e-03 -2.0446777e-03
  0.0000000e+00 -3.7841797e-03  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.6010742e-03 -3.2348633e-03 -3.5400391e-03
 -4.6997070e-03  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  5.5780857e+01  0.0000000e+00 -3.2958984e-03
  0.0000000e+00 -2.8076172e-03  0.0000000e+00 -3.9672852e-03
  0.0000000e+00 -4.0283203e-03 -3.9672852e-04 -6.5612793e-04
  0.0000000e+00  0.0000000e+00  0.0000000e+00 -6.7138672e-04
 -2.8686523e-03  0.0000000e+00  0.0000000e+00 -3.7231445e-03
  0.0000000e+00  0.00000

In [18]:
#take maximum
print((un_avgd['CanESM5_p2f2'].rsd.values-un_avgd_b['LBLRTM_p1f1'].rsd.values)[0,:,0].max())

55.780857


In [24]:
#remove the faulty column, then take the rsd global averages at TOA
#there is one column of 
un_avgd_b['LBLRTM_p1f1'] = un_avgd_b['LBLRTM_p1f1'].where(abs(un_avgd_b['LBLRTM_p1f1'].rsd-un_avgd['CanESM5_p2f2'].rsd).sel(level=TOA) < 50, other=np.nan)
un_avgd['CanESM5_p2f2'] = un_avgd['CanESM5_p2f2'].where(abs(un_avgd_b['LBLRTM_p1f1'].rsd-un_avgd['CanESM5_p2f2'].rsd).sel(level=TOA) < 50, other=np.nan)

(un_avgd_b['LBLRTM_p1f1'].rsd.values-un_avgd['CanESM5_p2f2'].rsd.values)[0,:,0]

un_avgd_b['LBLRTM_p1f1'] = un_avgd_b['LBLRTM_p1f1'].dropna('site')
un_avgd['CanESM5_p2f2'] = un_avgd['CanESM5_p2f2'].dropna('site')

In [25]:
(compute_global_mean(un_avgd['CanESM5_p2f2'])-compute_global_mean(un_avgd_b['LBLRTM_p1f1'])).rsd.sel(level=TOA)

<xarray.DataArray 'rsd' (expt: 18)>
array([-0.00167847, -0.00167847, -0.00167847, -0.00167847, -0.00167847,
       -0.00167847, -0.00167847, -0.00167847, -0.00167847, -0.00167847,
       -0.00167847, -0.00167847, -0.00167847, -0.00167847, -0.00167847,
       -0.00167847, -0.00167847, -0.00167847], dtype=float32)
Coordinates:
  * expt     (expt) object 'Present day (PD)' ... 'LGM'

The difference in globally averaged rsd at TOA drops from 0.5427551 to -0.00167847 after the column with difference 55.780857
W/m^2 is removed.